In [1]:
import argparse
import pathlib
import random

import numpy as np
import pandas as pd
from copairs import map
from copairs.matching import assign_reference_index

# check if in a jupyter notebook
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
import warnings

import pycytominer.aggregate

# Suppress all RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import itertools

#

In [2]:
def run_mAP_across_time(
    df: pd.DataFrame,
):
    """
    Run mAP across timepoints specifies and hardcoded columns for this data

    Parameters
    ----------
    df : pd.DataFrame
        An aggregated dataframe with metadata and features and temporal information

    Returns
    -------
    dict
        A dictionary of dataframes with the mAP results for each
        timepoint.
    """
    unique_timepoints = df.Metadata_Time.unique()
    dict_of_map_dfs = {}
    for timepoint in unique_timepoints:
        single_time_df = df.loc[df.Metadata_Time == timepoint]
        reference_col = "Metadata_reference_index"
        df_activity = assign_reference_index(
            single_time_df,
            "Metadata_treatment == 'DMSO CTL'",
            reference_col=reference_col,
            default_value=-1,
        )
        pos_sameby = ["Metadata_treatment", reference_col]
        pos_diffby = []
        neg_sameby = []
        neg_diffby = ["Metadata_treatment", reference_col]
        metadata = df_activity.filter(regex="Metadata")
        profiles = df_activity.filter(regex="^(?!Metadata)").values

        activity_ap = map.average_precision(
            metadata, profiles, pos_sameby, pos_diffby, neg_sameby, neg_diffby
        )

        activity_ap = activity_ap.query("Metadata_treatment != 'DMSO CTL'")
        activity_map = map.mean_average_precision(
            activity_ap, pos_sameby, null_size=1000000, threshold=0.05, seed=0
        )
        activity_map["-log10(p-value)"] = -activity_map["corrected_p_value"].apply(
            np.log10
        )
        # flatten the multi-index columns to make it easier to work with
        dict_of_map_dfs[timepoint] = activity_map
    return dict_of_map_dfs

In [3]:
data_file_path = pathlib.Path(
    "../../4.processing_profiled_features/data/preprocessed_data/live_cell_pyroptosis_wave1_first_time_norm_fs_agg.parquet"
).resolve(strict=True)
pathlib.Path("../results").mkdir(exist_ok=True)
aggregate_df = pd.read_parquet(data_file_path)
aggregate_df.reset_index(drop=True, inplace=True)
aggregate_df = aggregate_df[~aggregate_df.Metadata_serum.str.contains("NuSerum")]
aggregate_df.reset_index(drop=True, inplace=True)
print(aggregate_df.shape)
# drop rows with nan in features (non Metadata)
aggregate_df = aggregate_df.dropna(
    subset=aggregate_df.filter(regex="^(?!Metadata)").columns
)
print(aggregate_df.shape)
aggregate_df.head()

(1242, 688)
(1242, 688)


,Metadata_treatment,Metadata_Well,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,Metadata_treatment2_unit,Metadata_serum,...,Nuclei_Texture_InfoMeas2_BF_3_03_256,Nuclei_Texture_InverseDifferenceMoment_CL488_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CL561_3_01_256,Nuclei_Texture_InverseDifferenceMoment_DNA_3_00_256,Nuclei_Texture_InverseDifferenceMoment_GSDM_3_01_256,Nuclei_Texture_SumAverage_BF_3_03_256,Nuclei_Texture_SumAverage_CL488_3_01_256,Nuclei_Texture_SumAverage_CL561_3_03_256,Nuclei_Texture_SumAverage_GSDM_3_00_256,Metadata_cells_per_well
0,DMSO CTL,C05,DMSO,00,None,CTL,None,None,None,FBS,...,-0.266245,0.308959,0.580883,0.356271,0.424513,-0.265689,-0.239349,0.186214,-0.234910,18
1,DMSO CTL,C05,DMSO,01,None,CTL,None,None,None,FBS,...,-0.393354,0.394725,0.518643,-0.250707,0.542314,-0.558364,-0.275287,0.186214,-0.235567,18
2,DMSO CTL,C05,DMSO,02,None,CTL,None,None,None,FBS,...,-0.395359,0.278522,0.499880,-0.391898,0.538211,-0.602228,-0.245764,0.186214,-0.235567,18
3,DMSO CTL,C05,DMSO,03,None,CTL,None,None,None,FBS,...,-0.262421,0.336232,0.423368,-0.311942,0.532128,-0.370069,-0.267151,0.186214,-0.235324,18
4,DMSO CTL,C05,DMSO,04,None,CTL,None,None,None,FBS,...,-0.362028,0.325843,0.453277,-0.373401,0.499854,-0.400821,-0.255794,0.186214,-0.234708,18


In [4]:
# get the non metadata columns
metadata_cols = [cols for cols in aggregate_df.columns if "Metadata" in cols]
features_cols = [cols for cols in aggregate_df.columns if "Metadata" not in cols]
# drop features that contain AreaShape in the feature name
non_feature_cols = [cols for cols in features_cols if "AreaShape" in cols]
non_feature_cols = (
    non_feature_cols
    + [col for col in features_cols if "Location" in col]
    + [col for col in features_cols if "Neighbors" in col]
)
correlation_cols = [col for col in features_cols if "Correlation" in col]
features_cols = [cols for cols in features_cols if cols not in non_feature_cols]
features_cols = [cols for cols in features_cols if cols not in correlation_cols]
df = pd.DataFrame(features_cols, columns=["features"])

df[
    [
        "Compartment",
        "Feature_type",
        "Measurement",
        "Channel",
        "Feature_info1",
        "Feature_info2",
        "Feature_info3",
    ]
] = df["features"].str.split("_", expand=True)
df.head()

,features,Compartment,Feature_type,Measurement,Channel,Feature_info1,Feature_info2,Feature_info3
0,Cytoplasm_Granularity_1_BF,Cytoplasm,Granularity,1,BF,None,None,None
1,Cytoplasm_Granularity_1_CL488,Cytoplasm,Granularity,1,CL488,None,None,None
2,Cytoplasm_Granularity_1_CL561,Cytoplasm,Granularity,1,CL561,None,None,None
3,Cytoplasm_Granularity_1_DNA,Cytoplasm,Granularity,1,DNA,None,None,None
4,Cytoplasm_Granularity_1_GSDM,Cytoplasm,Granularity,1,GSDM,None,None,None


In [5]:
# create a dictionary of loadable features for each channel
loadable_features = {}
for channel in df.Channel.unique():
    channel_df = df.query("Channel == @channel")
    loadable_features[channel] = channel_df.features.tolist()
loadable_features["None"] = non_feature_cols
loadable_features["All"] = [
    cols for cols in aggregate_df.columns if "Metadata" not in cols
]

In [6]:
unique_channels = df["Channel"].unique().tolist()
# unique_channels = unique_channels + ['None']
# channel combinations
channel_combinations = []
for i in range(1, len(unique_channels) + 1):
    channel_combinations.extend(list(itertools.combinations(unique_channels, i)))
channel_combinations = [list(comb) for comb in channel_combinations]
channel_combinations = channel_combinations + [["None"]]

In [7]:
channel_combo_output_dict = {}
for shuffle in [True, False]:
    for channel_combination in channel_combinations:
        features_to_load = []
        features_to_load.append(metadata_cols)
        if len(channel_combination) == 5:
            channel_combination = ["All"]
        elif channel_combination == ["None"]:
            features_to_load.append(loadable_features["None"])
        else:
            for channel in channel_combination:
                features_to_load.append(loadable_features[channel])
            features_to_load.append(loadable_features["None"])

        features_to_load = list(itertools.chain(*features_to_load))
        temporary_df = aggregate_df[features_to_load]
        if shuffle:
            shuffle = "shuffled"
            random.seed(0)
            # permutate the data
            for col in temporary_df.columns:
                if "Metadata_Time" in col or "Metadata_treatment" in col:
                    pass
                else:
                    temporary_df[col] = np.random.permutation(temporary_df[col])
        else:
            shuffle = "non_shuffled"
        dict_of_map_dfs = run_mAP_across_time(temporary_df)
        df = pd.concat(dict_of_map_dfs.values(), keys=dict_of_map_dfs.keys())
        df.reset_index(inplace=True)
        df["Channel"] = "_".join(channel_combination)
        df["shuffle"] = shuffle
        channel_combo_output_dict["_".join(channel_combination)] = df

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

In [8]:
final_df = pd.concat(
    channel_combo_output_dict.values(), keys=channel_combo_output_dict.keys()
)
final_df.reset_index(inplace=True, drop=True)
final_df.rename(columns={"level_0": "Metadata_Time"}, inplace=True)
# save the output to a file
final_df.to_parquet("../results/mAP_across_channels.parquet")
final_df.head()

,Metadata_Time,Metadata_treatment,Metadata_reference_index,mean_average_precision,indices,p_value,corrected_p_value,below_p,below_corrected_p,-log10(p-value),Channel,shuffle
0,00,Ab1-42 0.4 uM,-1,0.700000,"[14, 43, 47]",2.998607e-01,0.471210,False,False,0.326786,BF,non_shuffled
1,00,Ab1-42 10 uM,-1,0.733333,"[18, 31, 62]",2.998607e-01,0.471210,False,False,0.326786,BF,non_shuffled
2,00,Ab1-42 2 uM,-1,1.000000,"[17, 33, 57]",9.999990e-07,0.000002,True,True,5.611820,BF,non_shuffled
3,00,Flagellin 0.1 ug/ml,-1,0.591667,"[1, 3, 9]",3.995696e-01,0.517090,False,False,0.286434,BF,non_shuffled
4,00,Flagellin 1 ug/ml,-1,0.575000,"[4, 20, 30]",4.993425e-01,0.578186,False,False,0.237932,BF,non_shuffled
